# Exercise 2.7 Subplots with cartopy
prepared by M.Hauser

We already learned how to create subplots in [Exercise 1.4](./../Part1_Matplotlib/ex1_4_subplots.ipynb), and we also used this for map plots. However, they suffer from the same problem as when adding colorbars - the axes are shrunk but the figure is not, leaving large gaps between the rows (or columns)...

> Most of what we show here for geo-referenced plots does *NOT* apply to normal subplots.

In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
import mplotutils as mpu

## Setup

Usually we want the background of the figure to be white - but here we want to see how large it really is. Therefore, we color it gray.

In [ ]:
%config InlineBackend.print_figure_kwargs = {'facecolor': '0.9'}

## The Problem - example

Let's do a 2 x 2 map plot:

In [ ]:
f, axs = plt.subplots(2, 2, subplot_kw=dict(projection=ccrs.PlateCarree()))
axs = axs.flatten()

for ax in axs:
    ax.coastlines()

There is too much vertical space between the subplots! Again, the issue is that the aspect ratio of a map plot has to be equal, else it would be distorted. Matplotlib then 'shrinks' the axes, but does not shrink the area of the figure.

## What we can try

There are some functions in pyplot that can potentially solve this problem. (However, they don't).


* _constrained layout_, automatically adjusts subplots and decorations like legends and colorbars so that they fit in the figure window while still preserving, as best they can, the logical layout requested by the user.
* _subplot_adjust_, i.e. setting the distance between the subplots manually

> There is also _tight layout_ with a similar goal but different approach as _constrained layout_. However, the latter generally seems to work better.

Before we can illustrate these two, we need to get another obstacle out of the way.


## `bbox_inches='tight'`

When you save a figure you can tell matplotlib to remove all white boundary areas using `plt.savefig('figure.pdf', bbox_inches='tight')`. While this sounds like a good idea, the issue is that the size of the figure is unpredictable. This can distort your font sizes. Also, if a journal asks for a figure with a certain size you cannot use this option.

> `bbox_inches='tight'` is NOT the same as `f.tight_layout()`

However, figures displayed in the notebooks use this option per default (in contrast to figures you save). To better see what figures we create, we need to turn this option off, and again, set the background color to a light gray.

In [ ]:
%config InlineBackend.print_figure_kwargs = {'bbox_inches': None, 'facecolor': '0.9'}

When we now redo the plot from above we see what our pdf would look like:

In [ ]:
f, axs = plt.subplots(2, 2, subplot_kw=dict(projection=ccrs.PlateCarree()))
axs = axs.flatten()

for ax in axs:
    ax.coastlines()
    ax.set_global()

So there is not only unequal distance between the figures, it also has a very large boundary.

## constrained layout

As mentioned _constrained layout_ tries to make sure all elements have space on the figure, but this does not really help either:

In [ ]:
f, axs = plt.subplots(
    2, 2, subplot_kw=dict(projection=ccrs.PlateCarree()), layout="constrained"
)
axs = axs.flatten()

for ax in axs:
    ax.coastlines()

### Different projections

While for the PlateCarree maps the figure was too high, it can also be too small for different projections:

In [ ]:
f, axs = plt.subplots(2, 2, subplot_kw=dict(projection=ccrs.Orthographic()))

axs = axs.flatten()

for ax in axs:
    ax.coastlines()

f.tight_layout()

## `subplot_adjust`

We already got to know `wspace` and `hspace` in [Exercise 1.4](../Part1_Matplotlib/ex1_4_subplots.ipynb#hspace-and-wspace). Using `subplot_adjust` we can manually fine tune the positioning of the subplots:

* `left`, `right`, `bottom`, and `top` are measured in figure coordinates, where (0, 0) is in the lower left corner and (1, 1) is in the upper right corner
* `wspace` is the amount of width reserved for blank space between subplots, expressed as a fraction of the average axis width, default value = 0.2
* `hspace` is the amount of height reserved for white space between subplots, expressed as a fraction of the average axis height, default value = 0.2

For a normal plot (i.e. non map plots), we can use them to adjust the figure to our liking: 

In [ ]:
f, axs = plt.subplots(2, 2)
axs = axs.flatten()

for ax in axs:
    ax.set_xticks([])
    ax.set_yticks([])

f.subplots_adjust(left=0.1, right=0.95, bottom=0.1, top=0.9, hspace=0.0, wspace=0.0)

Again, this does not work for axes with a fixed aspect ratio because matplotlib 'shrinks' the axes for map plots, setting hspace and wspace to 0 can leave some vertical distance between subplots. Therefore, the following plot still has some distance between the two rows:

In [ ]:
f, axs = plt.subplots(2, 2, subplot_kw=dict(projection=ccrs.PlateCarree()))
axs = axs.flatten()

for ax in axs:
    ax.coastlines()

f.subplots_adjust(hspace=0, wspace=0)

# Solution



To solve this problem we can use `mpu.set_map_layout`. This function respects all parameters of `subplot_adjust` (left, right, bottom, top, hspace, and wspace), and the figure width, but *NOT* its height. To be precise, it calculates the figure height such that all other parameters are respected.

#### Advantages
 * creating a map plot becomes predictable again (e.g. when you set `hspace=0`, the vertical space is actually 0)

#### Disadvantages
 * it only really works if all subplots have the same aspect ratio
 * needs manual adjustments (of subplot_adjust params)
 
#### Usage

1. Create figure and axes
1. Do all the plotting
1. use `f.subplots_adjust`
1. use `mpu.set_map_layout(axs, width=17)`
1. save the figure




In [ ]:
# 1. create figure
f, axs = plt.subplots(2, 2, subplot_kw=dict(projection=ccrs.PlateCarree()))
axs = axs.flatten()

# 2. plot stuff
for ax in axs:
    ax.coastlines()

# 3. adjust the position parameters
f.subplots_adjust(hspace=0, wspace=0)

# 4. call set_map_layout
mpu.set_map_layout(axs, width=17)  # width is in cm

print("Size (w x h):  ", np.round(f.get_size_inches() * 2.54, 2))

# 5. save
# plt.savefig(...)

It worked! `hspace=0, wspace=0` is actually respected!

### Exercise

* run the code below
* compare the width/ height of the figure below with the one above

In [ ]:
f, axs = plt.subplots(2, 2, subplot_kw=dict(projection=ccrs.PlateCarree()))
axs = axs.flatten()

for ax in axs:
    ax.coastlines()

f.subplots_adjust(hspace=0.3, wspace=0.15, left=0.05, right=0.95, bottom=0.05, top=0.95)

mpu.set_map_layout(axs, width=17)

print("Size (w x h):  ", np.round(f.get_size_inches() * 2.54, 2))

### Exercise

 * use `mpu.set_map_layout`, and `f.subplots_adjust` to create a 'good' looking figure

In [ ]:
f, axs = plt.subplots(2, 2, subplot_kw=dict(projection=ccrs.NearsidePerspective()))
axs = axs.flatten()

for i, ax in enumerate(axs):
    ax.coastlines()
    ax.set_title(f"Map {i}", fontsize=18)

# =============
# adjust size

# f.subplots_adjust(...)
mpu.set_map_layout(axs)

# =============

print("Size (w x h):  ", np.round(f.get_size_inches() * 2.54, 2))

### Solution

In [ ]:
f, axs = plt.subplots(2, 2, subplot_kw=dict(projection=ccrs.NearsidePerspective()))
axs = axs.flatten()

for i, ax in enumerate(axs):
    ax.coastlines()
    ax.set_title(f"Map {i}", fontsize=18)

# =============
# adjust size
f.subplots_adjust(hspace=0.2, wspace=0.1, left=0.05, right=0.95, bottom=0.05, top=0.925)

mpu.set_map_layout(axs)

# =============

print("Size (w x h):  ", np.round(f.get_size_inches() * 2.54, 2))

## Colorbars

`set_map_layout` can be combined with the manual creation of colorbars. You will have to manually make room for the colorbars, by adjusting left, bottom, etc...

In [ ]:
# create sample data
lon, lat, data = mpu.sample_data_map(90, 48)

opt = dict(vmin=-1.2, vmax=1.2, transform=ccrs.PlateCarree())

# =====

f, axs = plt.subplots(2, 2, subplot_kw=dict(projection=ccrs.PlateCarree()))
axs = axs.flatten()

for ax in axs:
    ax.coastlines()


f.subplots_adjust(hspace=0.2, wspace=0.25, left=0.025, right=0.9, bottom=0.05, top=0.95)

# ===================================

# color bar1
h = axs[0].pcolormesh(lon, lat, data, cmap="BrBG", **opt)
cbar = mpu.colorbar(h, axs[0], pad=0.015, size=0.05)

# color bar2
h = axs[2].pcolormesh(lon, lat, data, cmap="RdGy", **opt)
cbar = mpu.colorbar(h, axs[2], pad=0.015, size=0.05)

# color bar2
h = axs[1].pcolormesh(lon, lat, data, cmap="RdBu_r", **opt)
h = axs[3].pcolormesh(lon, lat, data, cmap="RdBu_r", **opt)
cbar = mpu.colorbar(h, axs[1], axs[3], pad=0.015, size=0.05)

# ===================================

mpu.set_map_layout(axs, width=17)

# ===================================

f.get_size_inches() * 2.54

### Exercise

 * take the plot below and make it look good

In [ ]:
f, axs = plt.subplots(1, 2, subplot_kw=dict(projection=ccrs.NearsidePerspective()))
axs = axs.flatten()

for i, ax in enumerate(axs):
    ax.coastlines()
    ax.set_title(f"Map {i}", fontsize=18)
    h = ax.pcolormesh(lon, lat, data, vmin=-1, vmax=1, transform=ccrs.PlateCarree())

# color bar1
cbar = mpu.colorbar(h, axs[1], size=0.1)

# =============
# adjust size

# f.subplots_adjust(...)

mpu.set_map_layout(axs)

# =============

print("Size (w x h):  ", np.round(f.get_size_inches() * 2.54, 2))

### Solution

In [ ]:
f, axs = plt.subplots(1, 2, subplot_kw=dict(projection=ccrs.NearsidePerspective()))
axs = axs.flatten()

for i, ax in enumerate(axs):
    ax.coastlines()
    ax.set_title(f"Map {i}", fontsize=18)
    h = ax.pcolormesh(lon, lat, data, vmin=-1, vmax=1, transform=ccrs.PlateCarree())

# color bar1
cbar = mpu.colorbar(h, axs[1], size=0.1)

f.subplots_adjust(hspace=0.2, wspace=0.1, left=0.05, right=0.85, bottom=0.05, top=0.85)

mpu.set_map_layout(axs, width=15.24)

print("Size (w x h):  ", np.round(f.get_size_inches() * 2.54, 2))

# Different projections

To combine different projections in subplots, or to combine map and none map plots, you have to use `gridspec`.

In [ ]:
import matplotlib.gridspec as gridspec

In [ ]:
gs = gridspec.GridSpec(2, 2)

ax1 = plt.subplot(gs[0, 0], projection=ccrs.PlateCarree())

ax2 = plt.subplot(gs[0, 1], projection=ccrs.Robinson())

ax3 = plt.subplot(gs[1, :])

ax1.coastlines()
ax2.coastlines()

x = np.arange(0, 10, 0.1)
ax3.plot(x, np.sin(x))
ax3.plot(x, np.cos(x))

Like this we can create a rotating earth:

In [ ]:
# n_col subplots
n_col = 10

f = plt.figure()
gs = gridspec.GridSpec(1, n_col)

axs = list()

# loop from 0...9
for i in range(n_col):

    # rotate
    c_lon = 360.0 / n_col * i

    # create subplot
    ax = plt.subplot(
        gs[i], projection=ccrs.Orthographic(central_longitude=c_lon, central_latitude=0)
    )

    # add some features
    ax.coastlines()
    ax.add_feature(cfeature.OCEAN)
    ax.add_feature(cfeature.LAND)

    # collect the axes
    axs.append(ax)

# adjust the size
f.subplots_adjust(wspace=0.1, left=0.025, right=0.975)
mpu.set_map_layout(axs, width=40)

# Bonus: Correct distance between subplots, alternative Solution

As for the colorbars, there is a second solution to overcome the problem, and again it is by using `axes_grid1`:

"[axes_grid1](https://matplotlib.org/2.0.2/mpl_toolkits/axes_grid/users/overview.html) is a collection of helper classes to ease displaying (multiple) images with matplotlib. In matplotlib, the axes location (and size) is specified in the normalized figure coordinates, which may not be ideal for displaying images that needs to have a given aspect ratio."

 > However, it is not part of the core matplotlib functionality, and not its best-documented part

We will use an [example from the cartopy gallery](http://scitools.org.uk/cartopy/docs/v0.15/examples/axes_grid_basic.html).

In [ ]:
from mpl_toolkits.axes_grid1 import AxesGrid
from cartopy.mpl.geoaxes import GeoAxes

The syntax is not super simple..., `axes_pad=[hspace, wspace]` defines the distance between the subplots in inches.

In [ ]:
projection = ccrs.PlateCarree()
axes_class = (GeoAxes, dict(projection=projection))


f = plt.figure()
axgr = AxesGrid(
    f,
    111,
    axes_class=axes_class,
    nrows_ncols=(3, 2),
    axes_pad=[2 / 2.54, 1 / 2.54],
    label_mode="",
)  # note the empty label_mode

axs = axgr.axes_all

for ax in axs:
    ax.coastlines()

plt.savefig("AxesGrid.pdf")

This may look wrong - but the distance between the subplots are exactly 2 cm & 1 cm (see the AxesGrid.pdf).

While hspace and wspace are correct, this comes at the cost that left, right, top, and bottom may not be set as specified.

### Exercise

 * set axes_pad to 0.5 cm
 * set the figure size to 16 cm x 13 cm (`f.set_size_inches`)
 * play around with `f.subplots_adjust` to find a good setting

In [ ]:
projection = ccrs.PlateCarree()
axes_class = (GeoAxes, dict(projection=projection))

f = plt.figure()

# set figure size
# f.set_size_inches(...)

axgr = AxesGrid(
    f,
    111,
    axes_class=axes_class,
    nrows_ncols=(3, 2),
    axes_pad=[2 / 2.54, 1 / 2.54],
    label_mode="",
)

axs = axgr.axes_all

for ax in axs:
    ax.coastlines()

# adjust the subplots
# f.subplots_adjust(left=0.1, right=0.9, bottom=0.1, top=0.9)

### Solution

In [ ]:
projection = ccrs.PlateCarree()
axes_class = (GeoAxes, dict(projection=projection))


f = plt.figure()

# set figure size
f.set_size_inches(16 / 2.54, 13 / 2.54)

axgr = AxesGrid(
    f,
    111,
    axes_class=axes_class,
    nrows_ncols=(3, 2),
    axes_pad=[0.5 / 2.54, 0.5 / 2.54],
    label_mode="",
)  # note the empty label_mode

axs = axgr.axes_all

for ax in axs:
    ax.coastlines()

f.subplots_adjust(left=0.05, right=0.95, bottom=0.05, top=0.95)

## Colorbars

You can also directly add colorbars.

In [ ]:
# create sample data
lon, lat, data = mpu.sample_data_map(90, 48)

# =====

f = plt.figure()

f.set_size_inches(16 / 2.54, 14.5 / 2.54)


projection = ccrs.PlateCarree()
axes_class = (GeoAxes, dict(projection=projection))

axgr = AxesGrid(
    f,
    111,
    axes_class=axes_class,
    nrows_ncols=(3, 2),
    axes_pad=[0.5 / 2.54, 0.5 / 2.54],
    cbar_location="bottom",
    cbar_mode="single",
    cbar_pad=0.1 / 2.54,
    cbar_size="10%",
    label_mode="",
)  # note the empty label_mode


axs = axgr.axes_all
for ax in axs:
    ax.coastlines()
    h = ax.pcolormesh(
        lon, lat, data, vmin=-1, vmax=1, cmap="RdBu_r", transform=ccrs.PlateCarree()
    )
    ax.set_global()

axgr.cbar_axes[0].colorbar(h)

f.subplots_adjust(left=0.05, right=0.95, bottom=0.1, top=0.95)

Note how the figure size and bottom had to be adjusted.

### Exercise

 * plot the same data on 3 subplots below each other
 * choose an Orthographic projection
 * add a colorbar for each plot, on the right side of the subplot
 * bonus: adjust the figure size and the subplot params 

In [ ]:
# uncomment to get the docstring
# AxesGrid?

In [ ]:
# create sample data
lon, lat, data = mpu.sample_data_map(90, 48)

# =====

# code here

### Solution

In [ ]:
# create sample data
lon, lat, data = mpu.sample_data_map(90, 48)

# =====

projection = ccrs.Orthographic()
axes_class = (GeoAxes, dict(projection=projection))

f = plt.figure()

f.set_size_inches(8 / 2.54, 14.5 / 2.54)

axgr = AxesGrid(
    f,
    111,
    axes_class=axes_class,
    nrows_ncols=(3, 1),
    axes_pad=0.75 / 2.54,
    cbar_location="right",
    cbar_mode="each",
    cbar_pad=0.5 / 2.54,
    cbar_size="10%",
    label_mode="",
)  # note the empty label_mode


axs = axgr.axes_all
for i, ax in enumerate(axs):
    ax.coastlines()
    h = ax.pcolormesh(
        lon, lat, data, vmin=-1, vmax=1, cmap="RdBu_r", transform=ccrs.PlateCarree()
    )

    axgr.cbar_axes[i].colorbar(h)


f.subplots_adjust(left=0.05, right=0.9, bottom=0.05, top=0.95)